In [4]:
%pip install transformers pandas

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.10.1
    Uninstalling fsspec-2021.10.1:
      Successfully uninstalled fsspec-2021.10.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipywidgets as widgets
from IPython.display import display
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import re

In [7]:
%pip install torch torchvision torchaudio

  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [5]:
# Set the pad_token_id to eos_token_id to avoid warnings
tokenizer.pad_token_id = tokenizer.eos_token_id

In [15]:
# Data cleansing function
def clean_text(text):
    if pd.isna(text):
        return ""
    # Removing unwanted characters like multiple commas, quotes, and extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

In [16]:
# Step to remove duplicate product names in the same record
def remove_duplicate_product_names(product_name):
    # Split the product name into words
    words = product_name.split()

    # Create an empty list to store the non-repeated words
    seen_words = []
    for word in words:
        # Add word to seen_words if it's not already added
        if ' '.join(seen_words).find(word) == -1:
            seen_words.append(word)
        else:
            break 

    # Join the unique words to recreate the cleaned product name
    return ' '.join(seen_words)

In [17]:
# Step 1: Load and preprocess the product names from CSV
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)  # Assuming the file is a CSV
    # Clean the dataset (remove duplicates, NaN values, etc.)
    df = df.dropna().drop_duplicates()

    # Apply cleaning to the 'Product Name' column
    df['Product Name'] = df['Product Name'].apply(clean_text)

    # Remove empty or invalid entries after cleaning
    df = df[df['Product Name'].str.len() > 0]
    
    # Apply the cleaning function to each product name in the dataframe
    df['Product Name'] = df['Product Name'].apply(remove_duplicate_product_names)
    product_names = df['Product Name'].tolist()  
    return product_names

# Load your dataset (replace 'product_names.csv' with your actual file path)
product_names = load_and_preprocess_data('dataset/product_names.csv')

In [19]:
# Step 2: Implement the retrieval system
def retrieve_similar(input_text, product_list, max_results=5):
    input_text_lower = input_text.lower()  
    retrieved_products = [name for name in product_list if input_text_lower in name.lower()]  
    return retrieved_products[:max_results]  

In [20]:
# Step 3: Generate text from retrieved products
def generate_completion(input_text, retrieved_products, max_new_tokens=30, num_return_sequences=1):
    # Join retrieved product names with a newline for better separation
    input_with_context = "\n".join(retrieved_products)

    # Tokenize input
#     input_ids = tokenizer.encode(input_with_context, return_tensors="pt")

    # Tokenize input with attention mask
    inputs = tokenizer(input_with_context, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']  
    
    
    # Generate auto-completion suggestions
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  
        max_new_tokens=max_new_tokens, 
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id 
    )

    # Decode and return generated text
    suggestions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Post-process suggestions: Split by newlines, remove duplicates, and clean up
    processed_suggestions = []
    for suggestion in suggestions:
        # Split the suggestion into product names
        product_names = suggestion.split("\n")

        # Deduplicate and clean up the product names
        unique_names = list(dict.fromkeys(product_names)) 

        # Join unique product names back together
        cleaned_suggestion = "\n".join(unique_names).strip() 

        processed_suggestions.append(cleaned_suggestion)

    return processed_suggestions

In [27]:
# If input text is not empty, retrieve suggestions and display
input_text = "fire tv"
if input_text:
    retrieved = retrieve_similar(input_text, product_names)  
    if retrieved:
        suggestions = generate_completion(input_text, retrieved)  
        # Update output box
#         suggestions_output.value = "\n\n".join(suggestions)
#         print(suggestions)
    else:
        suggestions_output.value = "No suggestions found."
        
for i in suggestions:
    print(i)

Certified Refurbished Amazon Fire TV Previous Generation - 1st
Certified Refurbished Amazon Fire TV Stick Previous Generation - 1st
Certified Refurbished Amazon Fire TV Stick Previous Generation - 1st Kindle Paperwhite
Amazon Echo and Fire TV Power Adapter
Certified Refurbished Amazon Fire TV with Alexa Voice Remote (AUR)
Certified Refurbished Amazon Fire TV


In [21]:
# Step 4: Update the UI to display suggestions dynamically
def on_text_change(change):
    # Get the input text
    input_text = change['new'].strip()

    # If input text is not empty, retrieve suggestions and display
    if input_text:
        retrieved = retrieve_similar(input_text, product_names)  
        if retrieved:
            suggestions = generate_completion(input_text, retrieved)  
            # Update output box
            suggestions_output.value = "\n\n".join(suggestions)
        else:
            suggestions_output.value = "No suggestions found."
    else:
        suggestions_output.value = "" 

# Create a text input box
text_input = widgets.Text(
    value='',
    placeholder='Type a product name...',
    description='Search:',
    disabled=False
)

# Create a text area for displaying suggestions
suggestions_output = widgets.Textarea(
    value='',
    placeholder='Suggestions will appear here...',
    description='Suggestions:',
    layout=widgets.Layout(width='80%', height='150px')
)

# Attach the text input box to a callback function to dynamically update suggestions
text_input.observe(on_text_change, names='value')

# Display the input box and output area in the notebook
display(text_input, suggestions_output)


Text(value='', description='Search:', placeholder='Type a product name...')

Textarea(value='', description='Suggestions:', layout=Layout(height='150px', width='80%'), placeholder='Sugges…